In [ ]:
import os
import pickle
import google.auth.transport.requests
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [ ]:
# 定义OAuth2.0范围
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

# 获取凭证
def get_credentials():
    creds = None
    # 如果存在token.pickle文件，加载之前保存的凭证
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # 如果没有凭证或凭证失效，重新获取
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(google.auth.transport.requests.Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # 保存新的凭证
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

# 获取Gmail服务
def get_gmail_service():
    creds = get_credentials()
    service = build('gmail', 'v1', credentials=creds)
    return service

# 获取邮件列表示例
service = get_gmail_service()
results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
messages = results.get('messages', [])
for message in messages:
    msg = service.users().messages().get(userId='me', id=message['id']).execute()
    print(msg['snippet'])
